In [1]:


import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import random

SERVICE_TIME = 5.0              # minutes per customer
TRUCK_SPEED = 50.0 / 60.0       # km per minute (approx. 0.833 km/min)
MAX_CAPACITY = 1300           # kilograms
NEW_VEHICLE_PENALTY = 10.0    # penalty for starting a new vehicle
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

def euclidean_distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)


# Environment for DSCVRPTW
class DSCVRPTWEnv:
    def __init__(self, data):
        """
        data: pandas DataFrame containing columns:
              'x', 'y', 'demand', 'open', 'close', 'time'
              Row 0 is the warehouse.
        """
        self.data = data.reset_index(drop=True)
        self.n_total = len(data)
        self.n_customers = self.n_total - 1  # excluding warehouse
        self.reset()
    
    def reset(self):
        self.time = 0.0  # simulation time in minutes
        # Vehicles: each vehicle is a dict with its current route, location, capacity, available time, and total distance traveled.
        warehouse = self.data.iloc[0]
        self.vehicles = [{
            'route': [0],
            'current_node': 0,
            'capacity': MAX_CAPACITY,
            'time': 0.0,
            'distance': 0.0
        }]
        # All customer indices (1..n_total-1) are initially unserved.
        self.unserved = set(range(1, self.n_total))
        # For dynamic arrivals, we keep track of which customers have “appeared.”
        self.available_customers = set()
        self.served = set()
        return self._get_state()
    
    def _get_state(self):
        """
        Returns a dictionary with:
            'time': current simulation time,
            'vehicles': list of vehicle states,
            'customers': list of customer dictionaries.
               (Each customer dict includes its original index and features.)
        """
        customers_state = []
        for idx in self.unserved:
            cust = self.data.iloc[idx].to_dict()
            cust['index'] = idx  # keep original index
            customers_state.append(cust)
        state = {
            'time': self.time,
            'vehicles': self.vehicles,
            'customers': customers_state
        }
        return state

    def update_dynamic_customers(self):
        """Update available customers based on simulation time and their appearance time."""
        for idx in list(self.unserved):
            cust_time = self.data.iloc[idx]['time']
            if self.time >= cust_time:
                self.available_customers.add(idx)
    
    def step(self, actions):
        """
        actions: list of tuples (vehicle_index, next_customer_index or None)
        For each vehicle, if an action is given, check feasibility (capacity, time window).
        Returns:
            next_state, reward, done
        """
        self.update_dynamic_customers()
        reward = 0.0
        
        # Process each vehicle's chosen action.
        for veh_idx, next_cust in actions:
            vehicle = self.vehicles[veh_idx]
            if next_cust is None:
                continue  # no action for this vehicle
            
            # Only serve if customer is available and unserved.
            if next_cust not in self.available_customers or next_cust not in self.unserved:
                continue
            
            cust_info = self.data.iloc[next_cust]
            # Capacity constraint
            if cust_info['demand'] > vehicle['capacity']:
                continue
            
            # Calculate travel time from current node to customer.
            current_node = vehicle['current_node']
            curr_info = self.data.iloc[current_node]
            distance = euclidean_distance(curr_info['x'], curr_info['y'], cust_info['x'], cust_info['y'])
            travel_time = distance * (60.0 / 50.0)  # 1.2 factor
            
            # Arrival time: if arriving before customer window, wait until open.
            arrival_time = max(vehicle['time'] + travel_time, cust_info['open'])
            if arrival_time > cust_info['close']:
                # Cannot serve due to time window violation.
                continue
            
            # Update vehicle state.
            vehicle['route'].append(next_cust)
            vehicle['capacity'] -= cust_info['demand']
            vehicle['time'] = arrival_time + SERVICE_TIME
            vehicle['distance'] += distance
            vehicle['current_node'] = next_cust
            
            # Mark customer as served.
            self.unserved.remove(next_cust)
            if next_cust in self.available_customers:
                self.available_customers.remove(next_cust)
            self.served.add(next_cust)
            
            # Accumulate reward (minimizing distance).
            reward -= distance
        
        # Advance global simulation time: here, we choose the minimum next available time among vehicles.
        self.time = min([veh['time'] for veh in self.vehicles])
        
        # If no current vehicle can serve any available customer, spawn a new vehicle.
        if self.unserved:
            can_serve = False
            for veh in self.vehicles:
                curr_info = self.data.iloc[veh['current_node']]
                for idx in self.unserved:
                    if idx not in self.available_customers:
                        continue
                    cust_info = self.data.iloc[idx]
                    # Check capacity and approximate time window feasibility.
                    if cust_info['demand'] > veh['capacity']:
                        continue
                    distance = euclidean_distance(curr_info['x'], curr_info['y'], cust_info['x'], cust_info['y'])
                    travel_time = distance * (60.0 / 50.0)
                    arrival_time = max(veh['time'] + travel_time, cust_info['open'])
                    if arrival_time <= cust_info['close']:
                        can_serve = True
                        break
                if can_serve:
                    break
            if not can_serve:
                # Start a new vehicle at the warehouse.
                warehouse = self.data.iloc[0]
                new_vehicle = {
                    'route': [0],
                    'current_node': 0,
                    'capacity': MAX_CAPACITY,
                    'time': self.time,
                    'distance': 0.0
                }
                self.vehicles.append(new_vehicle)
                reward -= NEW_VEHICLE_PENALTY
        
        # Check termination: all customers served.
        done = (len(self.unserved) == 0)
        next_state = self._get_state()
        return next_state, reward, done
    
    def render(self):
        for i, veh in enumerate(self.vehicles):
            print(f"Vehicle {i}: Route: {veh['route']}, Distance: {veh['distance']:.2f}, Remaining capacity: {veh['capacity']}")


# MARDAM-inspired Policy Network
class MARDAM(nn.Module):
    def __init__(self, customer_feature_dim, vehicle_feature_dim, embed_dim=128, num_heads=8, num_layers=2):
        """
        customer_feature_dim: number of features per customer (e.g., x, y, demand, open, close, appearance)
        vehicle_feature_dim: number of features per vehicle (e.g., current x, y, remaining capacity, current time)
        """
        super(MARDAM, self).__init__()
        # Customers encoding
        self.customer_encoder = nn.Linear(customer_feature_dim, embed_dim)
        # Vehicles encoding
        self.vehicle_encoder = nn.Linear(vehicle_feature_dim, embed_dim)
        # Transformer for contextualizing customer embeddings.
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # Turn focus – a learned parameter vector (could be further refined per turn)
        self.turn_focus = nn.Parameter(torch.randn(embed_dim))
        # Travels scorer – scores a vehicle-to-customer “travel option.”
        self.travel_scorer = nn.Linear(embed_dim * 2, 1)
    
    def forward(self, customer_features, vehicle_features):
        """
        customer_features: Tensor of shape [N, customer_feature_dim]
        vehicle_features: Tensor of shape [M, vehicle_feature_dim]
        Returns:
            probs: Tensor of shape [M, N] giving (softmax) probabilities over the N customers for each vehicle.
        """
        # Encode customers and pass through transformer.
        cust_emb = self.customer_encoder(customer_features)  # [N, embed_dim]
        # Transformer expects shape [sequence, batch, embed_dim] – we treat each customer as a “sequence element”
        cust_emb = cust_emb.unsqueeze(1)  # [N, 1, embed_dim]
        cust_emb = self.transformer(cust_emb)  # [N, 1, embed_dim]
        cust_emb = cust_emb.squeeze(1)  # [N, embed_dim]
        
        # Encode vehicles.
        veh_emb = self.vehicle_encoder(vehicle_features)  # [M, embed_dim]
        
        # For each vehicle and each customer, combine embeddings.
        M = veh_emb.size(0)
        N = cust_emb.size(0)
        veh_exp = veh_emb.unsqueeze(1).expand(M, N, -1)   # [M, N, embed_dim]
        cust_exp = cust_emb.unsqueeze(0).expand(M, N, -1)   # [M, N, embed_dim]
        combined = torch.cat([veh_exp, cust_exp], dim=-1)    # [M, N, 2*embed_dim]
        
        # Score travel options.
        scores = self.travel_scorer(combined).squeeze(-1)    # [M, N]
        # (Optionally, one could incorporate the turn focus here as an additive bias.)
        
        # Compute probability distributions (over customers) for each vehicle.
        probs = F.softmax(scores, dim=-1)  # [M, N]
        return probs


# Feature Extraction
def extract_features(state, data):
    """
    Given the current state (from env._get_state()) and original data,
    produce:
      - customer_features: tensor [N, 6] for each unserved customer: (x, y, demand, open, close, appearance time)
      - vehicle_features: tensor [M, 4] for each vehicle: (current x, current y, remaining capacity, current time)
      - mask: boolean tensor [N] indicating if a customer is available (appeared).
      - customers_list: list of customer dicts (including original index) corresponding to rows in customer_features.
    """
    vehicles = state['vehicles']
    vehicle_feats = []
    for veh in vehicles:
        curr_idx = veh['current_node']
        curr_node = data.iloc[curr_idx]
        # Vehicle features: current x, y, remaining capacity, current available time.
        vehicle_feats.append([curr_node['x'], curr_node['y'], veh['capacity'], veh['time']])
    vehicle_feats = torch.tensor(vehicle_feats, dtype=torch.float)
    
    customers_list = state['customers']
    cust_feats = []
    mask = []
    current_time = state['time']
    for cust in customers_list:
        # Features: x, y, demand, open, close, appearance time.
        cust_feats.append([cust['x'], cust['y'], cust['demand'], cust['open'], cust['close'], cust['time']])
        # A customer is “available” if the simulation time is past its appearance time.
        mask.append(current_time >= cust['time'])
    cust_feats = torch.tensor(cust_feats, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.bool)
    
    return cust_feats, vehicle_feats, mask, customers_list


# Training Loop
def train(model, env, data, n_episodes=100, lr=1e-3, gamma=1.0):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    for episode in range(n_episodes):
        state = env.reset()
        log_probs_episode = []
        rewards_episode = []
        done = False
        total_reward = 0.0
        
        while not done:
            # Extract features from current state.
            cust_feats, veh_feats, mask, customers_list = extract_features(state, data)
            # If no unserved customers remain, break.
            if cust_feats.size(0) == 0:
                break
            # Forward pass: get action probabilities.
            probs = model(cust_feats, veh_feats)  # shape [M, N]
            # Mask out customers that have not yet appeared.
            # mask is [N] – expand to each vehicle.
            mask_exp = mask.unsqueeze(0).expand_as(probs)
            probs = probs * mask_exp.float()
            # Avoid division by zero – renormalize.
            probs = probs / (probs.sum(dim=-1, keepdim=True) + 1e-10)
            
            actions = []
            step_log_probs = []
            # For each vehicle, sample an action.
            M, N = probs.shape
            for veh_idx in range(M):
                # Use .item() to convert tensor to scalar.
                if mask.sum().item() == 0:
                    # No available customer – choose no action.
                    actions.append((veh_idx, None))
                    step_log_probs.append(torch.tensor(0.0))
                else:
                    dist = Categorical(probs[veh_idx])
                    action_idx = dist.sample()
                    chosen_cust = customers_list[action_idx.item()]['index']
                    actions.append((veh_idx, chosen_cust))
                    step_log_probs.append(dist.log_prob(action_idx))
            
            next_state, reward, done = env.step(actions)
            total_reward += reward
            rewards_episode.append(reward)
            log_probs_episode.extend(step_log_probs)
            state = next_state
        
        # Compute (discounted) return; here gamma=1.0 so it is a sum.
        G = sum(rewards_episode)
        loss = 0.0
        for logp in log_probs_episode:
            loss -= logp * G  # REINFORCE loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Episode {episode+1}/{n_episodes} - Total Reward: {total_reward:.2f} - Loss: {loss.item():.4f} - Vehicles used: {len(env.vehicles)}")
    return model


# Main Routine
if __name__ == "__main__":
    # Load dataset from CSV with proper separator.
    data = pd.read_csv("F:/KLTN/100_Customer/h100c101.csv", sep=",", skipinitialspace=True)
    
    # (Optional) Inspect the first few rows.
    print("F:/KLTN/100_Customer/h100c101.csv")
    print(data.head())
    
    # Create environment.
    env = DSCVRPTWEnv(data)
    
    # Define model.
    # Customer feature dimension = 6 (x, y, demand, open, close, time)
    # Vehicle feature dimension = 4 (current x, current y, remaining capacity, current time)
    model = MARDAM(customer_feature_dim=6, vehicle_feature_dim=4, embed_dim=128, num_heads=8, num_layers=2)
    
    # Train the model.
    trained_model = train(model, env, data, n_episodes=50, lr=1e-3)
    
    # After training, render the final solution.
    final_state = env._get_state()
    env.render()


F:/KLTN/100_Customer/h100c101.csv
           x          y  demand  open  close  time
0  33.333333  41.666667       0     0   1236   0.0
1  37.500000  58.333333      30   825    870   0.0
2  35.000000  54.166667      10    15     67   0.0
3  18.333333  62.500000      30    30     92   0.0
4  25.000000  41.666667      10    10     73   0.0


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\admin\AppData\Local\Temp\ipykernel_884\1707775802.py:263: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  mask = torch.tensor(mask, dtype=torch.bool)


: 